In [217]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import macd_tv, ema_tv,rsi_tv,sma_tv
import pandas as pd
from datetime import datetime
import pandas as pd
import ta

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'ta' has no attribute '__version__'

In [197]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=1000,
    # since_datetime=datetime(2024, 3, 14),
    # until_datetime=datetime(2024, 3, 15),
)
datetimes = candles[:,CandleBodyType.Timestamp].astype('datetime64[ms]')
open_prices = candles[:,CandleBodyType.Open]
closing_prices = candles[:,CandleBodyType.Close]
low_prices = candles[:,CandleBodyType.Low]
high_prices = candles[:,CandleBodyType.High]
volume = candles[:,CandleBodyType.Volume]

In [5]:
def calculate_volume_color(open_price, close_price):
    if close_price >= open_price:
        return 'green'
    else:
        return 'red'

In [6]:
macd_below = 0
fast_length = 12
slow_length = 26
signal_smoothing = 9
ema_length = 200

In [7]:
ma_volume = sma_tv(
    source=volume,
    length=30
)

In [8]:
histogram, macd, signal = macd_tv(
    source=closing_prices,
    fast_length=fast_length,
    slow_length=slow_length,
    signal_smoothing=signal_smoothing
)

In [9]:
ema = ema_tv(
    source=closing_prices,
    length=ema_length,
)

In [10]:
prev_macd = np.roll(macd,1)
prev_macd[0] = np.nan

prev_signal = np.roll(signal,1)
prev_signal[0] = np.nan




In [11]:
rsi_is_below = 30
rsi_length = 14

In [12]:
rsi = rsi_tv(
    source=closing_prices,
    length=rsi_length
)

rsi = np.around(rsi,1)


In [13]:
macd_below_signal = prev_macd < prev_signal
macd_above_signal = macd > signal
low_price_below_ema = low_prices > ema
macd_below_number = macd < macd_below

In [241]:
def dirmov_tv(high, low, close, len):
    close_series = pd.Series(close)  # Convert to Pandas Series

    up = np.where(close_series - close_series.shift(1) > 0, close_series - close_series.shift(1), 0)
    down = np.where(close_series.shift(1) - close_series > 0, close_series.shift(1) - close_series, 0)

    plusDM = np.where((up > down) & (up > 0), up, 0)
    minusDM = np.where((down > up) & (down > 0), down, 0)

    truerange = np.maximum(high - low,
                           np.maximum(np.abs(high - close_series.shift(1)), 
                                  np.abs(low - close_series.shift(1))))

    plus = 100 * ema_tv(plusDM, len) / truerange
    minus = 100 * ema_tv(minusDM, len) / truerange

    return plus, minus

def adx_tv(high, low, close, di_length=14, adx_smoothing=14):
    plus, minus = dirmov_tv(high, low, close, di_length)
    sum = plus + minus
    adx = 100 * ema_tv(np.abs(plus - minus) / np.where(sum == 0, 1, sum), adx_smoothing)
    return adx



In [242]:

adx_values = adx_tv(high_prices, low_prices, closing_prices)
print(adx_values)
# print(datetimes[len(datetimes)-1])
# print(closing_prices[-5:]) 

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan 7.53 10.93 12.46 16.58 20.79 21.46 23.78
 23.38 22.54 25.00 28.06 27.57 26.97 27.76 29.26 28.97 27.49 26.37 23.13
 20.61 17.91 19.08 17.26 16.23 18.38 22.51 22.65 24.60 27.28 26.06 25.70
 25.63 25.23 24.37 22.99 21.54 19.38 19.49 19.87 21.10 22.50 22.46 20.53
 21.80 23.07 20.10 17.82 15.63 13.64 13.36 15.88 19.60 22.34 20.24 17.60
 15.60 17.24 15.58 13.57 12.60 16.49 20.25 24.82 29.70 27.93 27.91 27.15
 23.55 20.49 18.26 16.33 16.44 15.94 16.21 17.17 21.24 26.05 23.72 21.52
 20.85 19.19 17.50 18.43 16.60 15.85 16.90 16.02 14.18 14.78 14.57 15.81
 16.34 14.81 14.38 15.03 18.04 15.76 16.32 16.94 16.84 15.14 15.17 14.95
 16.52 16.29 17.03 18.45 22.20 19.49 17.33 15.44 15.84 15.74 16.18 16.13
 14.09 12.35 13.72 16.29 17.65 20.05 24.53 29.99 36.34 42.60 47.04 51.44
 54.55 53.20 53.02 51.15 51.36 53.81 53.55 54.75 56.02 57.91 53.48 46.61
 40.52 38.22 35.19 35.40 36.33 34.58 31.54 29.94 2

In [243]:
fig = go.Figure(data=[
    go.Candlestick(
        x=datetimes,
        open=open_prices, high=high_prices,
        low=low_prices, close=closing_prices
    ),
])
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [244]:
fig = go.Figure()
fig.add_scatter(
    x=datetimes,
    y=adx_values,
    name="adx",
    line_color="orange",
)

In [215]:
def has_recent_oversold_rsi(rsi_values, rsi_period, oversold_level=30):
    """Checks if there was any RSI value below the oversold level within a certain lookback period"""
    print('call')
    for i in range(rsi_period):
        print(rsi_values[-1 - i])
        if rsi_values[-1 - i] < oversold_level:  
            return True
    return False

In [216]:
look_back = 10
has_recent_oversold_rsi = np.zeros_like(closing_prices, dtype=bool)  # Initialize as False
for i in range(look_back, len(rsi)):
    window = rsi[i - look_back: i]
    has_recent_oversold_rsi[i] = np.any(window < rsi_is_below)    

In [217]:
volume_above_ma = volume > ma_volume

macd_cross = (
    # (low_price_below_ema == True)
    (macd_above_signal == True)
    & (macd_below_signal == True)
    & (macd_below_number == True)
    )

entries = (
    # (low_price_below_ema == True)
    (macd_above_signal == True)
    & (macd_below_signal == True)
    & (macd_below_number == True)
    &(has_recent_oversold_rsi == True)
    &(volume_above_ma == True)
    # & (has_recent_oversold_rsi(rsi, look_back, rsi_is_below))
    
)

In [218]:
entry_signals_rsi = np.where(entries, rsi, np.nan)
entry_signals_macd = np.where(entries, macd, np.nan)
macd_cross_plot = np.where(macd_cross, macd, np.nan)
entry_signals_close = np.where(entries, closing_prices, np.nan)


In [219]:
exit_prices = np.full_like(rsi, np.nan)

In [220]:
fig = go.Figure()

In [221]:
fig.add_scatter(
    x=datetimes,
    y=rsi,
    name="rsi",
    line_color="yellow",
)

In [222]:
fig.add_scatter(
    x=datetimes,
    y=entry_signals_rsi,
    mode="markers",
    name="entries",
    marker=dict(
        size=12,
        symbol="circle",
        color="#00F6FF",
        line=dict(
            width=1,
            color="DarkSlateGrey",
        ),
    ),
)

In [223]:
fig.add_hline(
    y=rsi_is_below,
    opacity=0.3,
    line_color="red",
)

In [224]:
fig = go.Figure()
fig = make_subplots(
    cols=1,
    rows=4,
    shared_xaxes=True,
    subplot_titles=["Candles","MACD","RSI"],
    # row_heights=[0.6,0.4],
    vertical_spacing=0.1
)
# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=datetimes,
        open=candles[:, CandleBodyType.Open],        
        high=candles[:, CandleBodyType.High],        
        low=candles[:, CandleBodyType.Low],        
        close=candles[:, CandleBodyType.Close],       
        name="Candles",
    ),
    col=1,
    row=1
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=ema,
        name="EMA",
        line_color="yellow",
    ),
    col=1,
    row=1,
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=entry_signals_close,
        mode="markers",
        name="entries",
        marker=dict(
            size=12,
            symbol="circle",
            color="#00F6FF",
            line=dict(
                width=1,
                color="DarkSlateGray",
            )
        )
    ),
    row=1,
    col=1        
)



ind_shift = np.roll(histogram, 1)
ind_shift[0] = np.nan
colors = np.where(
    histogram >= 0,
    np.where(ind_shift < histogram, "#26A69A", "#B2DFBD"),
    np.where(ind_shift < histogram, "#FFCDD2", "#FF5252"),
)

fig.append_trace(
    go.Bar(
        x=datetimes,
        y=histogram,
        name="histogram",
        marker_color=colors
    ),
    row=2,
    col=1
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=macd,
        name="macd",
        line_color="#2962FF"
    ),
    row=2,
    col=1
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=signal,
        name="signal",
        line_color="#FF6D00"
    ),
    row=2,
    col=1
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        # y=entry_signals_macd,
        y=macd_cross_plot,
        mode="markers",
        name="entries",
        marker=dict(
            size=12,
            symbol="circle",
            color="#00F6FF",
            line=dict(
                width=1,
                color="DarkSlateGray",
            )
        )
    ),
    row=2,
    col=1        
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=rsi,
        name="rsi",
        line_color="yellow",
    ),
    row=3,
    col=1
)

fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=entry_signals_rsi,
        mode="markers",
        name="entries",
        marker=dict(
            size=12,
            symbol="circle",
            color="#00F6FF",
            line=dict(
                width=1,
                color="DarkSlateGray",
            )
        )
    ),
    row=3,
    col=1        
)

fig.add_hline(
    y=rsi_is_below,
    opacity=0.3,
    line_color="red",
    row=3,
    col=1  
)

# Volume Indicator (Row 3)
fig.add_trace(go.Bar(
    x=datetimes, 
    y=volume,    
    name="Volume",
    marker_color=list(map(calculate_volume_color, open_prices, closing_prices))
), row=4, col=1)

fig.add_trace(go.Scatter(
    x=datetimes,
    y=ma_volume,
    mode='lines',
    name='Volume MA 30',
    line=dict(color='white')
), row=4, col=1) 


# Update options and show plot
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()
